In [1]:
import tensorflow as tf
import pickle
import unicodedata
import re
import numpy as np
import os
import io
import time
from util import *
from encoder import * 
from decoder import *
from train import *
from word_embedding import *
from recognition_nw import *
from prior_nw import *
from generation_nw import *

## Read data

In [2]:
PATH = '../data/'

In [3]:
d_tensor_test = np.load(PATH + 'd_tensor_test.npy',allow_pickle=True)
r_tensor_test = np.load(PATH + 'r_tensor_test.npy',allow_pickle=True)
dia_test = [t[0] for t in d_tensor_test]
aid_test = [t[1] for t in d_tensor_test]
res_test = [t[0] for t in r_tensor_test]
sid_test = [t[1] for t in r_tensor_test]

In [4]:
with open(PATH + 'tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
vocab_size = len(tokenizer.word_index) + 1

In [5]:
BATCH_SIZE = 64
ENC_HIDDEN_SIZE = 300
DEC_HIDDEN_SIZE = 512
NUM_LAYER = 4
DROP_OUT = 0.2
embedding_dim = 512
speaker_dim = 128
MAXLEN = 50
speakerNum = 14
EPOCHS = 10
LATENT_SIZE = 200
Use_Prior = True

## Response with beam search

In [6]:
checkpoint_dir = '../vae_ckpt_SM_300e_512d_200l/'

In [7]:
word_embedding = Word_Embedding(vocab_size, embedding_dim)
encoder = Encoder(ENC_HIDDEN_SIZE, vocab_size, BATCH_SIZE)
decoder = Decoder(DEC_HIDDEN_SIZE, vocab_size, speaker_dim, NUM_LAYER)
recognition_network = Recognition_Network(LATENT_SIZE)
prior_network = Prior_Network(LATENT_SIZE)
generation_network = Generation_Network(vocab_size, DEC_HIDDEN_SIZE)
optimizer = tf.keras.optimizers.Adam()

cp = tf.train.Checkpoint(optimizer=optimizer,
                             word_embedding=word_embedding,
                             encoder=encoder,
                             decoder=decoder,
                             recognition_network=recognition_network,
                             prior_network=prior_network,
                             generation_network=generation_network)
status = cp.restore(checkpoint_dir + "speaker-ckpt-4")
print(status)
train_nn = Train(word_embedding, encoder, decoder, recognition_network, prior_network, generation_network,optimizer,tokenizer)

In [8]:
def convert_to_word(response_sentence,tokenizer):
    sentence = list()
    for idx in response_sentence:
        sentence.append(tokenizer.index_word[idx])
        if(tokenizer.index_word[idx]=='<eos>'):
            break;
    print(' '.join(sentence))

In [9]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<sos> ' + w + ' <eos>'
    return w

In [10]:
def conver_to_tensor(sentence,tokenizer):
    sentence = preprocess_sentence(sentence)
    inputs = [tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=MAXLEN,
                                                         padding='post')
    tensor = tf.convert_to_tensor(inputs,dtype=np.int32)
    return tensor[0]

In [11]:
def response(train_nn,inp,speaker_id,addressee_id=None,max_len = MAXLEN,beam_size = 2):
    generate_size = beam_size
    inputs = np.expand_dims(inp, axis=0)
    s_id = np.expand_dims(speaker_id, axis=0)
    if addressee_id is not None:
        a_id = np.expand_dims(addressee_id, axis=0)
    inp_emb = train_nn.word_embedding(inputs)
    enc_out, enc_hidden,enc_c = train_nn.encoder(inp_emb)
    prior_mu, prior_logvar = train_nn.prior_network(enc_hidden)
    
    latent_sample = sample_gaussian(prior_mu, prior_logvar)
    dec_init_state, _ = train_nn.generation_network(enc_hidden, latent_sample)
    dec_input = train_nn.word_embedding(tf.convert_to_tensor([train_nn.tokenizer.word_index['<sos>']]))
#     print(dec_input)
    dec_input = tf.expand_dims(dec_input,0)
#     print(dec_input.shape)
    sos = train_nn.tokenizer.word_index['<sos>']
    
    if addressee_id is not None:
        predictions, dec_hidden, dec_c,_ = train_nn.decoder(dec_input,enc_out, dec_init_state,s_id,a_id)
    else:
        predictions, dec_hidden, dec_c,_ = train_nn.decoder(dec_input,enc_out, dec_init_state,s_id)

    dec_init_state = [dec_hidden,dec_c]
    pred_prob,pred_top_k = tf.math.top_k(predictions,k=beam_size)
#     for i in range(pred_prob[0].shape[0]):
#         print("prob:{},word:{}".format(pred_prob[0][i],tokenizer.index_word[pred_top_k[0][i].numpy()]))
    # beam: (prob,sentence,dec_inp,dec_init,isEnd)
    finished = list() # to store finished sentence
    beam = list()
    for ind, i in enumerate(pred_top_k[0]):
        word_idx = i.numpy()
        if tokenizer.index_word[word_idx]=='<eos>':
            finished.append([pred_prob[0][ind],[sos]+[word_idx]])
        i_emb = train_nn.word_embedding(tf.expand_dims(i,0))
#         print(i_emb.shape)
        beam.append([pred_prob[0][ind],[sos]+[i.numpy()],i_emb,dec_init_state,False])
    for t in range(2,max_len):
        tmp_beam = list()
        for ind,b in enumerate(beam):
            if b[-1]:
                continue
            dec_input = tf.expand_dims(b[2],1)
            dec_init_state = b[3]
            if addressee_id is not None:
                predictions, dec_hidden, dec_c,_ = train_nn.decoder(dec_input,enc_out, dec_init_state,s_id,a_id)
            else:
                predictions, dec_hidden, dec_c,_ = train_nn.decoder(dec_input,enc_out, dec_init_state,s_id)
            
            dec_init_state = [dec_hidden,dec_c]

            # use beam search to get top k prediction
            pred_prob,pred_top_k = tf.math.top_k(predictions,k=beam_size)
#             for i in range(pred_prob[0].shape[0]):
#                 print("beam:{} prob:{},word:{}".format(ind,pred_prob[0][i],tokenizer.index_word[pred_top_k[0][i].numpy()]))
            beam_prob = beam[ind][0]
            beam_sen = beam[ind][1]
            for ind, i in enumerate(pred_top_k[0]):
                isEnd = False
                word_idx = i.numpy()
                if tokenizer.index_word[word_idx]=='<eos>':
                    isEnd = True
                i_emb = train_nn.word_embedding(tf.expand_dims(i,0))
                tmp_beam.append([(beam_prob + pred_prob[0][ind])/(t+1),beam_sen + [word_idx],i_emb,dec_init_state,isEnd])
                
        # select top k candidates combination
        tmp_beam.sort(key=lambda x:x[0],reverse=True)
        tmp_beam = tmp_beam[:generate_size]
        beam.clear()
        for ind, b in enumerate(tmp_beam):
            if b[-1]:
                finished.append(b[:2])
                generate_size = generate_size - 1
            else:
                tmp_beam[ind][0] = b[0] * (t+1) # continue to compute probability of whole sentence
                beam.append(tmp_beam[ind])
                
    finished.sort(key=lambda x:x[0],reverse=True)
    print("{} Finished sentence:".format(len(finished)))
    for i in range(len(finished)):
        convert_to_word(finished[i][1],train_nn.tokenizer)
#     response_sentence = finished[:beam_size]
#     print("Response sentence:")
#     for i in range(beam_size):
#         convert_to_word(beam[i][1],train_nn.tokenizer)
    

In [12]:
idx = 5645

In [22]:
convert_to_word(dia_test[idx],tokenizer)
convert_to_word(res_test[idx],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx],sid_test[idx],beam_size=3)
print("addressee:{}".format(aid_test[idx]))
print("speaker:{}".format(sid_test[idx]))

<sos> oh . so , how many children do you think we should have ? i m sorry , that was a little abrupt . <eos>
<sos> a little . <eos>
RESPONSE:
3 Finished sentence:
<sos> okay , let s go . <eos>
<sos> okay , let s just talk about the rest of my life . <eos>
<sos> okay , let s just talk about the rest of my business . <eos>
addressee:13
speaker:6


In [27]:
speaker = tf.convert_to_tensor(7)
convert_to_word(dia_test[idx],tokenizer)
convert_to_word(res_test[idx],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx],speaker,beam_size=3)

<sos> oh . so , how many children do you think we should have ? i m sorry , that was a little abrupt . <eos>
<sos> a little . <eos>
RESPONSE:
3 Finished sentence:
<sos> i know . <eos>
<sos> i m sorry . what are you doing ? <eos>
<sos> i m sorry . <eos>


In [34]:
speaker = tf.convert_to_tensor(8)
convert_to_word(dia_test[idx],tokenizer)
convert_to_word(res_test[idx],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx],speaker,beam_size=3)

<sos> oh . so , how many children do you think we should have ? i m sorry , that was a little abrupt . <eos>
<sos> a little . <eos>
RESPONSE:
3 Finished sentence:
<sos> yeah , well , i ve been thinking about you guys . <eos>
<sos> yeah , well , i ve been thinking about it . <eos>
<sos> yeah , well , i ve been thinking about you . <eos>


In [35]:
speaker = tf.convert_to_tensor(9)
convert_to_word(dia_test[idx],tokenizer)
convert_to_word(res_test[idx],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx],speaker,beam_size=3)

<sos> oh . so , how many children do you think we should have ? i m sorry , that was a little abrupt . <eos>
<sos> a little . <eos>
RESPONSE:
3 Finished sentence:
<sos> what are you doing ? <eos>
<sos> what ? <eos>
<sos> really ? <eos>


In [36]:
speaker = tf.convert_to_tensor(10)
convert_to_word(dia_test[idx],tokenizer)
convert_to_word(res_test[idx],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx],speaker,beam_size=3)

<sos> oh . so , how many children do you think we should have ? i m sorry , that was a little abrupt . <eos>
<sos> a little . <eos>
RESPONSE:
3 Finished sentence:
<sos> really ? <eos>
<sos> oh , that s so sweet . <eos>
<sos> oh , that s great . <eos>


In [44]:
speaker = tf.convert_to_tensor(11)
convert_to_word(dia_test[idx],tokenizer)
convert_to_word(res_test[idx],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx],speaker,beam_size=3)

<sos> oh . so , how many children do you think we should have ? i m sorry , that was a little abrupt . <eos>
<sos> a little . <eos>
RESPONSE:
3 Finished sentence:
<sos> i thought you liked it . <eos>
<sos> you know , i was wondering if you were a kid . <eos>
<sos> you know , i was wondering if you were a kid , too . <eos>


In [66]:
idx2 = 3000

In [65]:
for i in range(3000,3200):
    print(i)
    convert_to_word(dia_test[i],tokenizer)

3000
<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
3001
<sos> if anyone asks , we ll just say ben addressed them . oh ! so you invited rachel then ? <eos>
3002
<sos> okay . <eos>
3003
<sos> what was monica s nickname when she was a field hockey goalie ? <eos>
3004
<sos> hey , hey . <eos>
3005
<sos> she does want to . <eos>
3006
<sos> screw him . you re fine . <eos>
3007
<sos> all right . leonard went to the office . <eos>
3008
<sos> oh my god , i can t believe what i m getting ready to say . i wanna have a baby , but i don t wanna have one with someone who doesn t really wanna have one . <eos>
3009
<sos> so does that answer your question ? <eos>
3010
<sos> talk to her ? that s all you ve got ? with a cool name like sheldon cooper s council of ladies , i really expected more . give me back the t shirts . <eos>
3011
<sos> it s her i have to worry about . <eos>
3012
<sos> fine , i

In [67]:
convert_to_word(dia_test[idx2],tokenizer)
convert_to_word(res_test[idx2],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx2],sid_test[idx2],beam_size=3)
print("addressee:{}".format(aid_test[idx2]))
print("speaker:{}".format(sid_test[idx2]))

<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
<sos> hey , no matter what happens with ross and emily , we still get cake right ? <eos>
RESPONSE:
3 Finished sentence:
<sos> well , i don t think so . <eos>
<sos> well , i don t know . <eos>
<sos> well , i don t think so . i mean , you know , i think you re gonna be able to be able to be a little uncomfortable . <eos>
addressee:13
speaker:0


In [69]:
speaker = tf.convert_to_tensor(1)
convert_to_word(dia_test[idx2],tokenizer)
convert_to_word(res_test[idx2],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx2],speaker,beam_size=3)

<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
<sos> hey , no matter what happens with ross and emily , we still get cake right ? <eos>
RESPONSE:
4 Finished sentence:
<sos> hey ! <eos>
<sos> okay , you re a big deal ! <eos>
<sos> okay , you re a big deal . <eos>
<sos> <eos>


In [70]:
speaker = tf.convert_to_tensor(2)
convert_to_word(dia_test[idx2],tokenizer)
convert_to_word(res_test[idx2],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx2],speaker,beam_size=3)

<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
<sos> hey , no matter what happens with ross and emily , we still get cake right ? <eos>
RESPONSE:
3 Finished sentence:
<sos> what ? ! <eos>
<sos> what ? <eos>
<sos> what are you doing ? ! <eos>


In [72]:
speaker = tf.convert_to_tensor(3)
convert_to_word(dia_test[idx2],tokenizer)
convert_to_word(res_test[idx2],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx2],speaker,beam_size=3)

<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
<sos> hey , no matter what happens with ross and emily , we still get cake right ? <eos>
RESPONSE:
3 Finished sentence:
<sos> well , i mean , i mean , i mean , i mean , i mean , you re the only one who s gonna be able to get married . <eos>
<sos> well , i mean , i mean , it s not like i ve been thinking about it . <eos>
<sos> well , i mean , i mean , i mean , i mean , i mean , you re the only one who s gonna be able to get married , and i just have to be the one who s been married to . <eos>


In [73]:
speaker = tf.convert_to_tensor(4)
convert_to_word(dia_test[idx2],tokenizer)
convert_to_word(res_test[idx2],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx2],speaker,beam_size=3)

<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
<sos> hey , no matter what happens with ross and emily , we still get cake right ? <eos>
RESPONSE:
3 Finished sentence:
<sos> what ? ! <eos>
<sos> what ? <eos>
<sos> what ? ! what ? ! <eos>


In [76]:
speaker = tf.convert_to_tensor(5)
convert_to_word(dia_test[idx2],tokenizer)
convert_to_word(res_test[idx2],tokenizer)
print("RESPONSE:")
response(train_nn,dia_test[idx2],speaker,beam_size=3)

<sos> you ve spoiled everything ! it s like a nightmare ! my friends and family are out there ! how can i face them ? ! how can you do this to me ? ! <eos>
<sos> hey , no matter what happens with ross and emily , we still get cake right ? <eos>
RESPONSE:
3 Finished sentence:
<sos> i know . <eos>
<sos> oh , i m sorry . <eos>
<sos> i m sorry . <eos>
